In [1]:
import pyspark as ps
from pyspark.ml.evaluation import RegressionEvaluator
from pyspark.ml.recommendation import ALS
import pyspark.sql.functions as F
from pyspark.sql.types import StructType, StructField
from pyspark.sql.types import IntegerType, StringType, FloatType, DateType, TimestampType
import pyspark.sql.functions as F
from pyspark.mllib.evaluation import RegressionMetrics
# from MatrixFactorizationModel

VBox()

Starting Spark application


ID,YARN Application ID,Kind,State,Spark UI,Driver log,Current session?
15,application_1582822548125_0018,pyspark,idle,Link,Link,✔


FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

SparkSession available as 'spark'.


FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [62]:
user_test = spark.read.csv('s3://myanimelist-recommender/test_user_profile.csv', header = True)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [2]:
# user_test.show()
user_test = user_test.withColumn("anime_id", user_test.anime_id.cast(IntegerType()))
user_test = user_test.withColumn("my_score", user_test.my_score.cast(IntegerType()))
user_test= user_test.withColumn("user_id", user_test.user_id.cast(IntegerType()))
user_test = user_test.withColumn("stats_mean_score", user_test.stats_mean_score.cast(FloatType()))
# data = data.withColumn("my_watched_episodes", data.my_score.cast(IntegerType()))
# data = data.withColumn("my_status", data.my_score.cast(IntegerType()))
user_test

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

name 'user_test' is not defined
Traceback (most recent call last):
NameError: name 'user_test' is not defined



In [22]:
# This is data iteration one: it is compprised of 21994566 rows × 8 columns (blank, anime_id, username, user_id, my_score, my_watched_episodes, my_status)
# my status 1 - 3 only were used because they indicate 1: watching 2: completed 3: on hold 4: dropped 6: plan to watch
# so only 1-3  were what i considered really relevant, as droppers probably dont have fully formed opinions

# my_watched episodes was another metric i used to choose who to drop. anybody who had watched less than
# 3 episodes of a show likely did not have what i would consider a fully formed opinion so they were dropped.

# Furthermore, a large number of these made up the 0 ratings which were in overabundance
data = spark.read.csv('s3://myanimelist-recommender/final_anime.csv', header = True)
data.show()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+---+--------+--------+--------+-------+----------------+-------------------+---------+
|_c0|anime_id|username|my_score|user_id|stats_mean_score|my_watched_episodes|my_status|
+---+--------+--------+--------+-------+----------------+-------------------+---------+
|  0|      21|karthiga|       9|2255153|            7.43|                586|        1|
|  1|      59|karthiga|       7|2255153|            7.43|                 26|        2|
|  2|      74|karthiga|       7|2255153|            7.43|                 26|        2|
|  3|     120|karthiga|       7|2255153|            7.43|                 26|        2|
|  4|     178|karthiga|       7|2255153|            7.43|                 26|        2|
|  5|     210|karthiga|       7|2255153|            7.43|                161|        2|
|  6|     232|karthiga|       6|2255153|            7.43|                 70|        2|
|  7|     233|karthiga|       6|2255153|            7.43|                 78|        2|
|  8|     249|karthiga|       8|

In [5]:
data.count()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

15546712

In [2]:
# data iteration 2: it is one with similar conditions to the above, with the addition of
# filtering out any user or anime with less than 5 reviews. This will hopefully increase the 
# overall effectiveness of the model despite removing data
data = spark.read.csv('s3://myanimelist-recommender/final_anime_2.csv', header = True)
data.count()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

15546712

In [3]:
data = data.withColumn("anime_id", data.anime_id.cast(IntegerType()))
data = data.withColumn("user_id", data.user_id.cast(IntegerType()))
data = data.withColumn("my_score", data.my_score.cast(IntegerType()))
data = data.withColumn("stats_mean_score", data.stats_mean_score.cast(FloatType()))
# data = data.withColumn("my_watched_episodes", data.my_score.cast(IntegerType()))
# data = data.withColumn("my_status", data.my_score.cast(IntegerType()))

# data = data.drop('_c0')
data.show()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+---+--------+--------+-------+----------------+
|_c0|anime_id|my_score|user_id|stats_mean_score|
+---+--------+--------+-------+----------------+
|  0|      21|       9|2255153|            7.43|
|  1|      59|       7|2255153|            7.43|
|  2|      74|       7|2255153|            7.43|
|  3|     120|       7|2255153|            7.43|
|  4|     178|       7|2255153|            7.43|
|  5|     210|       7|2255153|            7.43|
|  6|     232|       6|2255153|            7.43|
|  7|     233|       6|2255153|            7.43|
|  8|     249|       8|2255153|            7.43|
|  9|     269|      10|2255153|            7.43|
| 10|     721|       7|2255153|            7.43|
| 11|     853|       7|2255153|            7.43|
| 12|     857|      10|2255153|            7.43|
| 13|     957|       8|2255153|            7.43|
| 14|     966|      10|2255153|            7.43|
| 15|    1557|       8|2255153|            7.43|
| 16|    1571|       8|2255153|            7.43|
| 17|    1579|      

In [2]:
# This is data iteration three (final_anime_3.csv). In this, users and anime have been further filtered
# only users who have rated >6 and anime that have been rated >6 times are included in this dataset. 
# otherwise it is the same as iteration 2. 
data = spark.read.csv('s3://myanimelist-recommender/anime_final_3.csv', header = True)
data.count()


VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

15022090

In [13]:
data = data.withColumn("anime_id", data.anime_id.cast(IntegerType()))
data = data.withColumn("user_id", data.user_id.cast(IntegerType()))
data = data.withColumn("my_score", data.my_score.cast(IntegerType()))
data = data.withColumn("stats_mean_score", data.stats_mean_score.cast(FloatType()))
# data = data.withColumn("my_watched_episodes", data.my_score.cast(IntegerType()))
# data = data.withColumn("my_status", data.my_score.cast(IntegerType()))

# data = data.drop('_c0')
# data = data.drop('my_watched_episodes')
data = data.drop( 'my_status')
data.show()


VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+---+--------+--------+--------+-------+----------------+
|_c0|anime_id|username|my_score|user_id|stats_mean_score|
+---+--------+--------+--------+-------+----------------+
|  0|      21|karthiga|       9|2255153|            7.43|
|  1|      59|karthiga|       7|2255153|            7.43|
|  2|      74|karthiga|       7|2255153|            7.43|
|  3|     120|karthiga|       7|2255153|            7.43|
|  4|     178|karthiga|       7|2255153|            7.43|
|  5|     210|karthiga|       7|2255153|            7.43|
|  6|     232|karthiga|       6|2255153|            7.43|
|  7|     233|karthiga|       6|2255153|            7.43|
|  8|     249|karthiga|       8|2255153|            7.43|
|  9|     269|karthiga|      10|2255153|            7.43|
| 10|     721|karthiga|       7|2255153|            7.43|
| 11|     853|karthiga|       7|2255153|            7.43|
| 12|     857|karthiga|      10|2255153|            7.43|
| 13|     957|karthiga|       8|2255153|            7.43|
| 14|     966|

In [7]:
training, testing = data.randomSplit([0.7, 0.3])

# training = training.drop('_c0')
# testing = testing.drop('_c0')
training.count()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

10882696

In [8]:
print("users in training: {}".format(training.select('user_id').distinct().count()))
print("users in testing: {}".format(testing.select('user_id').distinct().count()))

print("users in both: {}".format(training.select('user_id').distinct()\
                                         .join(testing.select('user_id').distinct(),
                                               'user_id', 'inner')\
                                         .count()))

print("movies in training: {}".format(training.select('anime_id').distinct().count()))
print("movies in testing: {}".format(testing.select('anime_id').distinct().count()))

print("movies in both: {}".format(training.select('anime_id').distinct()\
                                         .join(testing.select('anime_id').distinct(),
                                               'anime_id', 'inner')\
                                         .count()))

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

users in training: 106415
users in testing: 105586
users in both: 105516
movies in training: 3644
movies in testing: 3582
movies in both: 3555

In [14]:
class MovieRecommender_loop1():
    """Loop 1 recommender (see notebooks/solns.ipynb)
    Uses ALS out of the box"""
    def fit(self, train_df):
        self.als = ALS(rank=250,
              maxIter=20,
              regParam=0.1,
              userCol="user_id",
              itemCol="anime_id",
              ratingCol="my_score")

        self.loop1_model = self.als.fit(train_df)

    def transform(self, test_df):
        return(self.loop1_model.transform(test_df))


class MovieRecommender_loop2():
    """Loop 2 recommender (see notebooks/solns.ipynb)
    Uses average per movie to fill up NaNs left by ALS"""
    def fit(self, training_df):
        self.avg_ratings = training_df.select('anime_id','my_score', 'stats_mean_score')\
                                      .groupBy('anime_id')\
                                      .agg(F.avg('my_score'))\
                                      .withColumnRenamed('avg(my_score)','avg_rating')

    def transform(self, requests_df):
        return(requests_df.join(self.avg_ratings, 'anime_id', 'left')\
                          .withColumnRenamed('avg_rating','prediction'))


class AnimeRecommender(ALS):
    """Aggregated results from loop1 and loop2 (by coaleste)"""
    def __init__(self):
        self.spark = ps.sql.SparkSession.builder \
                    .master("local[4]") \
                    .appName("df lecture") \
                    .getOrCreate()
        self.sc = self.spark.sparkContext  # for the pre-2.0 sparkContext
        self.mr1 = MovieRecommender_loop1()
        self.mr2 = MovieRecommender_loop2()


    def fit(self, ratings):
        """
        Trains the recommender on a given set of ratings.
        Parameters
        ----------
        ratings : pandas dataframe, shape = (n_ratings, 4)
                  with columns 'user', 'movie', 'rating', 'timestamp'
        Returns
        -------
        self : object
            Returns self.
        """
        

#         self.training = self.spark.createDataFrame(ratings)
        self.training = ratings

        self.training.persist()

        self.mr1.fit(self.training)
        self.mr2.fit(self.training)

#         self.logger.debug("finishing fit")
        return(self)


    def transform(self, requests):
        """
        Predicts the ratings for a given set of requests.
        Parameters
        ----------
        requests : pandas dataframe, shape = (n_ratings, 2)
                  with columns 'user', 'movie'
        Returns
        -------
        dataframe : a *pandas* dataframe with columns 'user', 'movie', 'rating'
                    column 'rating' containing the predicted rating
        """

#         self.requests = self.spark.createDataFrame(requests)
        self.requests = requests

        pred_loop1 = self.mr1.transform(self.requests).withColumnRenamed('prediction','prediction_loop1')
        print(pred_loop1.show())
        pred_loop2 = self.mr2.transform(pred_loop1).withColumnRenamed('prediction','prediction_loop2')
# [anime_id, my_last_updated, user_id, prediction_loop2, prediction_loop1, my_score
        print(pred_loop2.show())
    
        results_loop2 = pred_loop2.withColumn('prediction',F.when(F.isnan('prediction_loop1'),F.col('prediction_loop2')).otherwise(F.col('prediction_loop1')))
#         results_loop2 = pred_loop2.withColumn('prediction',F.when(F.isnan('prediction_loop1'),F.col('stats_mean_score')).otherwise(F.col('prediction_loop1')))
        print(results_loop2.show())
        predictions = results_loop2.select('user_id', 'anime_id', 'my_score', 'prediction')\
                                   .withColumnRenamed('prediction','rating')

        return(predictions)
    
    def evaluate(self, predictions):
        evaluator = RegressionEvaluator(metricName="rmse", labelCol="my_score",predictionCol="rating")
        
        rmse = evaluator.evaluate(preds)
        return print(f'the rmse is {rmse}')

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [ ]:
mr = AnimeRecommender()
x = mr.fit(training)
preds = mr.transform(testing)


VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

An error was encountered:
Invalid status code '404' from https://172.31.16.159:18888/sessions/15/statements/15 with error payload: {"msg":"Session '15' not found."}


In [11]:
# preds = preds.withColumn("rating", preds.rating.cast(FloatType()))
print(preds.count())
preds = preds.dropna(subset=('rating'))
print(preds.count())
preds.show()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

4664016
4663996
+-------+--------+--------+------------------+
|user_id|anime_id|my_score|            rating|
+-------+--------+--------+------------------+
| 222730|     148|       8| 7.647821426391602|
|   6825|     148|       7| 6.761627674102783|
|  46781|     148|       7| 6.452243328094482|
| 120976|     148|       9| 7.875185012817383|
| 168196|     148|       8| 5.920736312866211|
| 265959|     148|       7| 7.037248134613037|
|2650907|     148|      10| 8.471237182617188|
| 155228|     148|       8| 7.836144924163818|
|   5926|     148|       0|1.0436757802963257|
|  75439|     148|       7| 5.825883388519287|
| 128607|     148|       5| 6.407979965209961|
| 204921|     148|       7| 6.358741760253906|
|   9029|     148|       9| 6.950005531311035|
|  38098|     148|       9|  6.64668083190918|
| 164515|     148|       6| 6.551913261413574|
|  18819|     148|       7| 7.085896015167236|
| 182092|     148|       8| 7.478772163391113|
|  56880|     148|       6|   6.510864257812

In [12]:
# cvgrid is not working, so ill do tuning by hand in the next cells
evaluator = RegressionEvaluator(metricName="rmse", labelCol="my_score",predictionCol="rating")
        
rmse = evaluator.evaluate(preds)


VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [81]:
# for anime_final: see comments above
# rmse = 1.8450456842750507 on initial pass of data set, rank = default, maxIter = default, my_status <3, minimum watched >0
# ^unrecreatable for some reason
# rmse = 2.1858678093919712 on another pass, rank = 100, maxIter = 10, reg param = .1
# rmse = 2.175129033735969 on rank = 250, maxIter = 10, regparam = .1
# rmse =  2.1712464062761363 on rank 100, maxIter = 15, regparam = .1
# rmse = 2.6971824770818507 on rank 100, maxIter = 10, reparam = .01
# rmse = 2.1791087286348882, rank 120, maxiter = 11, regparam =.1

rmse

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

2.1800007704960653

In [13]:
# for anime_final_2: see comments at top
# rmse = 2.1254230685892352 rank = 100, maxIter = 10, reg param = .1
# rmse = 2.1058600851187363 rank = 100, maxiter = 15, regparam =.1
# rmse = 2.0987042601431383 rank = 150, maxiter = 15, regparam = .1
# rmse = 1.6232912469115457, rank 200, maxiter 17, regparam .1 <----- huge win, saved these train splits
# rmse = 1.6143999936595663, rank 220, maxiter 18, reparam = .1 <------ bigger win, used a different split^
# rmse = rank 250, maxiter 20, regoaram .1
rmse

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

1.6143999936595663

In [24]:
# for final_anime_3: see comments at top
# rmse =1.9578590462477  rank 200, maxiter 17, regparam .1
# rmse = 2.0000903889660986, rank 100, maxiter 10, regparam .1
rmse

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

2.0000903889660986

In [15]:
training=training.repartition(1)
training.write.save("s3://myanimelist-recommender/final_anime_2_test.csv", format='csv', header=True)
testiing = testing.repartition(1)
testing.write.save("s3://myanimelist-recommender/final_anime_2_train.csv", format='csv', header=True)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [18]:
preds.select("anime_id").distinct().show()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+--------+
|anime_id|
+--------+
|     148|
|     463|
|     471|
|     496|
|     833|
|    1088|
|    1238|
|    1580|
|    1591|
|    1829|
|    1959|
|    2142|
|    2366|
|    3918|
|    4101|
|    5300|
|    6336|
|    6658|
|    8086|
|    9465|
+--------+
only showing top 20 rows

In [ ]:
model.save([spark_context], [file_path])
